In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import osmnx as ox
import geopandas as gpd
import overpy

PATH_DATA = "..\data"
pd.set_option('display.max_columns', 60)

api = overpy.Overpass()

In [194]:
vkusvill_data = pd.read_csv(os.path.join(PATH_DATA, "vkusvill_data_without_target.csv"))
vkusvill = vkusvill_data.copy()

# vkusvill_target = pd.read_csv(os.path.join(PATH_DATA, "vkusvill_target.csv"))
# vkusvill = pd.merge(vkusvill_data, vkusvill_target, on="Номерточки", how="left")

In [222]:
# Список объектов
# https://wiki.openstreetmap.org/wiki/RU:%D0%9E%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D1%8B_%D0%BA%D0%B0%D1%80%D1%82%D1%8B

tags = [
    {'highway': 'bus_stop'},

    {'building': 'apartments'},
    {'building': 'detached'},
    {'building': 'dormitory'},
    {'building': 'house'}, 
    {'building': 'residential'},
    {'building': 'semidetached_house'}, 

    {'building': 'commercial'},
    {'building': 'kiosk'}, 
    # {'building': 'office'},
    {'building': 'retail'},
    {"building": "supermarket"},
    {'building': 'train_station'},

    {'railway': 'station'},  
    {'railway': 'subway_entrance'},  
    {'railway': 'tram_stop'}, 

    {'amenity': 'bar'}, 
    {'amenity': 'cafe'}, 
    {'amenity': 'fast_food'}, 
    {'amenity': 'food_court'}, 
    {'amenity': 'pub'}, 
    {'amenity': 'restaurant'}, 

    {'amenity': 'college'},
    {'amenity': 'driving_school'},  
    {'amenity': 'language_school'},  
    {'amenity': 'school'},  
    {'amenity': 'kindergarten'}, 
    {'amenity': 'university'}, 

    {'amenity': 'car_wash'},
    {"amenity": "fuel"},

    {'amenity': 'atm'},  
    {'amenity': 'bank'},  

    {'amenity': 'clinic'}, 
    {'amenity': 'dentist'}, 
    {'amenity': 'doctors'}, 
    {'amenity': 'hospital'},  
    {'amenity': 'pharmacy'},
    {'amenity': 'veterinary'}, 
    
    {'amenity': 'theatre'},  
    {'amenity': 'cinema'},  
            
    {'tourism': 'hostel'},
    {'tourism': 'hotel'},
    ]

In [223]:
len(tags)

41

In [224]:
R = 1000

for tag in tqdm(tags, leave=len(tags)):
    key = list(tag.keys())[0]
    value = list(tag.values())[0]
    vkusvill[value] = None
    
    for idx, row in vkusvill.iterrows():
        data = api.query(f"""
        <query type="node">
            <around lat="{row["Широта"]}" lon="{row["Долгота"]}" radius="{R}"/>
            <has-kv k="{key}" v="{value}"/>
        </query>
        <print/>
        """)
        vkusvill.loc[idx, value] = len(data.nodes)

100%|██████████| 41/41 [40:11<00:00, 58.82s/it]


In [225]:
vkusvill['office'] = None
vkusvill['shop'] = None

for idx, row in tqdm(vkusvill.iterrows(), leave=vkusvill.shape[0]):
    office = api.query(f"""
    <query type="node">
        <around lat="{row["Широта"]}" lon="{row["Долгота"]}" radius="{R}"/>
        <has-kv k="office"/>
    </query>
    <print/>
    """)
    vkusvill.loc[idx, 'office'] = len(office.nodes)

    shop = api.query(f"""
    <query type="node">
        <around lat="{row["Широта"]}" lon="{row["Долгота"]}" radius="{R}"/>
        <has-kv k="shop"/>
    </query>
    <print/>
    """)
    vkusvill.loc[idx, 'shop'] = len(shop.nodes)

298it [02:21,  2.11it/s]


In [250]:
vkusvill.shape

(298, 54)

In [251]:
for col in vkusvill.columns:
    if vkusvill[col].nunique() == 1:
        vkusvill.drop(col, axis=1, inplace=True)

In [252]:
vkusvill.shape

(298, 46)

In [253]:
vkusvill

,Наименование,Номерточки,Дата открытия,"Торговая площадь, м2",Регион,Город,Адрес,Широта,Долгота,График,Ночной магазин,fuel,bus_stop,house,kiosk,retail,station,subway_entrance,tram_stop,bar,cafe,fast_food,food_court,pub,restaurant,college,driving_school,language_school,school,kindergarten,university,car_wash,atm,bank,clinic,dentist,doctors,hospital,pharmacy,veterinary,theatre,cinema,hostel,hotel,office,shop
0,4548М_Нах_Белобородова6,4548,44491,52.6,Московская область,Нахабино,"ул. Белобородова, д. 6",55.850328,37.192615,с 9-00 до 22-00,Нет,0,8,0,1,0,0,0,0,0,2,5,0,0,1,0,0,0,0,1,0,4,0,2,2,2,1,0,5,0,0,0,0,0,0,91
1,3297М_Некрас_мкрСтроителей42,3297,43799,78.0,Московская область,Некрасовский,"мкр. Строителей, д. 42",56.131445,37.504663,с 9-00 до 22-00,Нет,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,9
2,1013М_МалГрузинская12,1013,42600,73.0,Москва,Москва,Малая Грузинская 12,55.763797,37.572064,с 8-00 до 22-00,Нет,1,61,0,11,0,3,7,0,16,75,37,1,5,40,1,1,2,1,4,2,5,16,26,20,17,6,2,25,3,2,0,5,7,46,390
3,2530М_Мыт_Юбилейная4,2530,43661,85.5,Московская область,Мытищи,"ул. Юбилейная, д. 4",55.915306,37.720106,с 8-00 до 22-00,Нет,1,45,0,0,0,0,0,0,2,7,8,1,1,6,0,0,1,0,5,0,2,5,9,5,3,5,2,14,0,0,1,1,1,17,184
4,1473М_Хим_Ленинский1,1473,43201,100.0,Московская область,Химки,"Ленинский проспект, 1к2",55.895762,37.452660,с 8-00 до 22-00,Нет,0,36,0,11,1,1,0,0,3,18,22,0,2,6,0,3,0,0,0,0,3,8,9,2,9,6,0,30,2,0,0,0,1,22,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,1308М_Лоб_Ленина67,1308,44494,370.0,Московская область,Лобня,"г. Лобня, ул. Ленина д.67",56.008319,37.449659,с 9-00 до 22-00,Нет,0,21,0,0,0,0,0,0,0,3,1,0,0,3,0,2,0,0,0,0,2,3,2,5,2,3,0,5,0,1,0,0,0,10,78
294,5348М_Ист_Ленина65,5348,44446,310.0,Московская область,Истра,"ул. Ленина, дом № 65",55.912244,36.859008,с 7-00 до 23-00,Нет,0,20,0,0,0,0,0,0,1,16,19,0,2,4,0,2,1,0,0,0,0,13,7,3,2,7,0,15,1,1,1,0,0,31,205
295,3072М_БольшаяСпасская8,3072,43901,367.7,Москва,Москва,"ул. Большая Спасская , д. 8, стр. 1А",55.775839,37.644171,с 7:00 до 23:45,Да,1,43,0,13,0,9,23,19,13,86,72,3,4,49,0,1,2,1,2,1,4,45,38,18,11,5,0,28,1,6,0,10,16,143,411
296,3903М_Красн_Подмосковный2,3903,43976,371.9,Московская область,Красногорск,"б-р Подмосковный, д. 2 + ""ВкусВиллАйс""",55.819236,37.365523,с 7:00 до 23:45,Да,0,27,0,0,0,0,0,0,4,24,10,0,3,6,0,0,5,0,2,0,6,8,5,11,9,6,0,29,1,0,0,3,2,10,278


In [3]:
data = api.query("""
<query type="node">
    <around lat="55.850328" lon="37.192615" radius="1000"/>
    <has-kv k="highway" v="bus_stop"/>
</query>
<print/>
""")
	
print(len(data.nodes))
data.nodes[:5]
	

8


[<overpy.Node id=1562240193 lat=55.8446528 lon=37.2018870>,
 <overpy.Node id=1562240196 lat=55.8463281 lon=37.1945991>,
 <overpy.Node id=4366502294 lat=55.8556128 lon=37.1800351>,
 <overpy.Node id=4543592850 lat=55.8439299 lon=37.2037753>,
 <overpy.Node id=4545617568 lat=55.8459640 lon=37.1951641>]

In [254]:
vkusvill.to_csv(os.path.join(PATH_DATA, "vkusvill_new_features.csv"), index=False)